# **Colab**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
file_exportdb = '/content/drive/My Drive/Colab/files/v-2/item_nfe_classe_export_2025-03-15.csv'
file_distinto = '/content/drive/My Drive/Colab/files/v-2/item_nfe_classe_distinto.csv'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install unidecode
from unidecode import unidecode
import pandas as pd
import csv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 9.5 MB/s eta 0:00:00


In [ ]:
# Leitura do arquivo CSV, ignorando a primeira linha de cabeçalho
df = pd.read_csv(file_exportdb, skiprows=1, header=None, quotechar='"', delimiter=',')
# Definição das colunas originais
df.columns = ['categoria', 'ano', 'num_chave_nota_fiscal_eletronica', 'num_item_nota_fiscal_eletronica', 'cod_anp', 'cod_ncm', 'cod_unidade', 'des_anp', 'des_produto']
df = df[['categoria', 'cod_anp', 'cod_ncm', 'cod_unidade', 'des_anp', 'des_produto']]

In [ ]:
# Tratamento de valores nulos
df['cod_anp'] = df['cod_anp'].fillna('0')
df['cod_ncm'] = df['cod_ncm'].fillna('0')
df['des_anp'] = df['des_anp'].fillna('nihil')
df['des_anp'] = df['des_anp'].replace(['0', 'ANP'], 'nihil')
df['des_produto'] = df['des_produto'].fillna('nihil')
df['cod_unidade'] = df['cod_unidade'].fillna('desconhecida')

# Converção de 'cod_anp' e 'cod_ncm' para valores numéricos
df['cod_anp'] = df['cod_anp'].apply(lambda x: int(float(x)) if x != '' else 0)
df['cod_ncm'] = df['cod_ncm'].apply(lambda x: int(float(x)) if x != '' else 0)

# Remover aspas invertidas e substituir vírgulas por hífens no campo 'des_anp'
df['des_anp'] = df['des_anp'].str.replace(r'\\', '', regex=True)
df['des_anp'] = df['des_anp'].str.replace(',', '-')

# Remover aspas invertidas e substituir vírgulas por hífens no campo 'des_produto'
df['des_produto'] = df['des_produto'].str.replace(r'\\', '', regex=True)
df['des_produto'] = df['des_produto'].str.replace(',', '-')

# Converter para minúsculas e remover acentuação
df['des_anp'] = df['des_anp'].str.lower().apply(unidecode)
df['des_produto'] = df['des_produto'].str.lower().apply(unidecode)
df['cod_unidade'] = df['cod_unidade'].str.lower().apply(unidecode)

In [ ]:
len(df)

3695

In [ ]:
df['categoria'].value_counts()

,count
categoria,
CB,2116
XX,1579


In [ ]:
df['cod_ncm'].value_counts()

,count
cod_ncm,
27101259,1017
27101921,569
27101932,330
31021010,175
84212300,167
...,...
19049000,1
34070010,1
85124010,1


In [ ]:
df['cod_anp'].value_counts()

,count
cod_anp,
0,2158
320102001,503
820101034,258
320102002,233
820101033,119
620505001,118
810101001,97
320101001,27
820101012,20


In [ ]:
df['cod_unidade'].value_counts()

,count
cod_unidade,
un,1411
l,1121
lt,747
pc,122
unid,79
litro,76
und,41
unidad,16
uni,15


In [ ]:
# Padroniza os códigos de unidade
mapa_cod_unidade = {
    'u': 'un',
    'un1': 'un',
    'und': 'un',
    'uni': 'un',
    'unid': 'un',
    'unidad': 'un',
    'l': 'lt',
    'l.': 'lt',
    'lts': 'lt',
    'lit': 'lt',
    'litro': 'lt',
    'h': 'hr',
    'hs': 'hr',
    'pç': 'pc',
    'peca': 'pc'
}
df['cod_unidade'] = df['cod_unidade'].replace(mapa_cod_unidade)
df['cod_unidade'] = df['cod_unidade'].replace(mapa_cod_unidade)

In [ ]:
df['cod_unidade'].value_counts()

,count
cod_unidade,
lt,1949
un,1564
pc,137
hr,17
dl,8
gl,5
cx,3
cj,3
serv,2


In [ ]:
# Identificar valores com frequência abaixo de 10 para cod_ncm
low_freq_cod_ncm = df['cod_ncm'].value_counts()
low_freq_cod_ncm = low_freq_cod_ncm[low_freq_cod_ncm < 10].index

# Substituir esses valores por 0
df['cod_ncm'] = df['cod_ncm'].apply(lambda x: 0 if x in low_freq_cod_ncm else x)

# Identificar valores com frequência abaixo de 10 para cod_anp
low_freq_cod_anp = df['cod_anp'].value_counts()
low_freq_cod_anp = low_freq_cod_anp[low_freq_cod_anp < 10].index

# Substituir esses valores por 0
df['cod_anp'] = df['cod_anp'].apply(lambda x: 0 if x in low_freq_cod_anp else x)

# Identificar valores com frequência abaixo de 5 para cod_unidade
low_freq_cod_unidade = df['cod_unidade'].value_counts()
low_freq_cod_unidade = low_freq_cod_unidade[low_freq_cod_unidade < 5].index

# Substituir esses valores por 0
df['cod_unidade'] = df['cod_unidade'].apply(lambda x: 0 if x in low_freq_cod_unidade else x)

In [ ]:
df['cod_unidade'].value_counts()

,count
cod_unidade,
lt,1949
un,1564
pc,137
hr,17
0,15
dl,8
gl,5


In [ ]:
df['cod_anp'].value_counts()

,count
cod_anp,
0,2270
320102001,503
820101034,258
320102002,233
820101033,119
620505001,118
810101001,97
320101001,27
820101012,20


In [ ]:
df['cod_ncm'].value_counts()

,count
cod_ncm,
27101259,1017
27101921,569
0,396
27101932,330
31021010,175
84212300,167
22071090,120
22011000,115
84219999,110


In [ ]:
import nltk
from nltk.corpus import stopwords

# Baixar stopwords
nltk.download('stopwords')
# Obter a lista de stopwords em português
stopwords_portugues = list(stopwords.words('portuguese'))

# Exibir a quantidade de stopwords
#print("Quantidade de stopwords em português:", len(stopwords_portugues))
# Exibir algumas stopwords
#print("Algumas stopwords:", stopwords_portugues[:10])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# Função para filtrar palavras irrelevantes (stopwords)
def remover_stopwords(texto, stopwords_lista):
    if pd.isnull(texto):  # Verificar se o valor não é nulo
        return texto
    palavras = texto.split()
    palavras_filtradas = [palavra for palavra in palavras if palavra not in stopwords_lista]
    return ' '.join(palavras_filtradas)

In [ ]:
df1 = df.copy()
# Remover palavras irrelevantes (stopwords)
df1['des_anp'] = df1['des_anp'].apply(lambda x: remover_stopwords(x, stopwords_portugues))
df1['des_produto'] = df1['des_produto'].apply(lambda x: remover_stopwords(x, stopwords_portugues))

In [ ]:
# Filtrar apenas registros distintos após ajustes nas colunas categóricas
df2 = df1.drop_duplicates(subset=['categoria', 'cod_anp', 'cod_ncm', 'cod_unidade', 'des_anp', 'des_produto'])

In [ ]:
len(df2)

1911

In [ ]:
df2['cod_ncm'].value_counts()

,count
cod_ncm,
0,328
27101932,259
27101921,236
27101259,220
84212300,118
84219999,87
22011000,80
31021010,74
34029039,41


In [ ]:
df2['cod_anp'].value_counts()

,count
cod_anp,
0,1434
820101034,106
620505001,88
320102001,76
320102002,74
820101033,43
810101001,32
740101006,15
620501001,14


In [ ]:
df2['cod_unidade'].value_counts()

,count
cod_unidade,
un,1194
lt,615
pc,73
0,13
hr,10
gl,5
dl,1


In [ ]:
# Salvar o DataFrame filtrado em um arquivo CSV
df2.to_csv(file_distinto, index=False, quoting=csv.QUOTE_NONE, escapechar='\\')

In [ ]:
df2['categoria'].value_counts()

,count
categoria,
XX,1107
CB,804


In [ ]:
df2['cod_unidade'].value_counts()

,count
cod_unidade,
un,1194
lt,615
pc,73
0,13
hr,10
gl,5
dl,1


In [ ]:
df2['cod_anp'].value_counts()

,count
cod_anp,
0,1434
820101034,106
620505001,88
320102001,76
320102002,74
820101033,43
810101001,32
740101006,15
620501001,14


In [ ]:
df2['cod_ncm'].value_counts()

,count
cod_ncm,
0,328
27101932,259
27101921,236
27101259,220
84212300,118
84219999,87
22011000,80
31021010,74
34029039,41
